In [ ]:
# Python version
import sys
print( "Running Python" , sys.version )

In [ ]:
# Importing necessary modules
import os
import re
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Load google drive

from google.colab import drive
drive.mount( '/content/gdrive' )

# Case study: Postnatal maternal sleep data 2023

Case study analysis of maternal sleep data for postnatal months 2, 3, 4 and 5.

Maternal falling-asleep and waking-up times were recorded between 7th May and 11th September, 2023, inclusive, for a total of 128 days.

Baby slept in bedside crib next to maternal bed and was lifted out for feeding/comforting during wake periods in the night. No changes occurred to the daily or nightly routines during the time period of measurement.

# Definitions:

*   **Total sleep duration:** Sum total time asleep during a 24-hour period, measured from 6 pm on a given day to 6 pm on following day.
*   **Sleep interval**: A continuous, uninterrupted period of sleep, as measured between a consecutive start_time and end_time pair. There could be a number of sleep intervals in a single night or a 24-hour period.
*   **Sleep interval duration:** Duration of an individual sleep interval.
*   **Bedtime:** First time of falling asleep in a 24-hour period (6pm - 6pm).
*   **Month 2:** 7th May - 11th June (36 days)
*   **Month 3:** 12th June - 11th July (30 days)
*   **Month 4:** 12th July - 11th August (31 days)
*   **Month 5:** 12th August - 11th September (31 days)




In [ ]:
# Import the csv data

df = pd.read_csv( 'gdrive/MyDrive/Colab Notebooks/projects/dss/dss_csv.csv' )


# Convert data type of start_time and end_time columns to datetime64 (Timestamp) object

df[ 'start_time' ] = pd.to_datetime( df[ 'start_time' ] )
df[ 'end_time' ] = pd.to_datetime( df[ 'end_time' ] )


# Create daylabel column for grouping the times for one night based on the day on which it starts

df[ 'daylabel' ] = df[ 'start_time' ].apply( lambda t: t.date() )

# Create monthlabel column to indicate which postnatal month the date is in (month2, month3, etc.)

def label_month( t ):
  if t < pd.Timestamp( '2023-06-12 18:00:00' ):
    return 'month2'

  elif t >= pd.Timestamp( '2023-06-12 18:00:00' ) and t < pd.Timestamp( '2023-07-12 18:00:00' ):
    return 'month3'

  elif t >= pd.Timestamp( '2023-07-12 18:00:00' ) and t < pd.Timestamp( '2023-08-12 18:00:00' ):
    return 'month4'
  return 'month5'

df[ 'monthlabel' ] = df[ 'start_time' ].apply( label_month )


# Fix dates by shifting to next day when midnight is passed

df[ 'start_time' ] = df[ 'start_time' ].apply( lambda t: t + pd.Timedelta( days=1 ) if t.hour <= 18 and t.hour >= 0 else t )
df[ 'end_time' ] = df[ 'end_time' ].apply( lambda t: t + pd.Timedelta( days=1 ) if t.hour <= 18 and t.hour >= 0 else t )


# Create sleep duration column for each sleep interval

df[ 'sleep_dur' ] = df[ 'end_time' ] - df[ 'start_time' ]
df[ 'sleep_dur_mins' ] = df[ 'sleep_dur' ].apply( lambda td : int( td.seconds / 60 ) )
df[ 'sleep_dur_hrs' ] = df[ 'sleep_dur' ].apply( lambda td : td.seconds / 3600 )

# Create a number-of-sleep-cycles column for each sleep interval

df[ 'num_cycles' ] = df[ 'sleep_dur_mins' ].apply( lambda x : int( x / 90 ) )
df.head( 10 )


# Key results

**1) Total sleep duration**

Over the whole time period measured, total sleep duration per 24 hours had:
* median = 6.5 hours
* minimum = 2.2 hours
* maximum = 9.5 hours

The distribution was slightly negatively skewed.

Median total sleep duration changed over the months:
* Month 2: median = 5.9 hours
* Month 3: median = 6.4 hours
* Month 4: median = 7.0 hours
* Month 5: median = 6.3 hours

**2) Number of sleep intervals**

Descriptive statistics for number of sleep intervals per 24 hours:
* mode = 2 (i.e., one interruption to sleep)
* median = 3 (i.e., two interruptions to sleep)
* minimum = 1 (i.e., no interruptions to sleep)
* maximum = 6 (i.e., five interruptions to sleep)

This varied by month:
* Month 2: median = 4 sleep intervals (i.e., three interruptions to sleep)
* Month 3: median = 2 sleep intervals
* Month 4: median = 2 sleep intervals
* Month 5: median = 2 sleep intervals

**3) Sleep interval durations**

Median sleep interval durations overall:

* Month 2: 1.2 hours
* Month 3: 2.0 hours
* Month 4: 3.2 hours
* Month 5: 2.3 hours

A full adult sleep cycle is taken to be of length 90 minutes.

The proportion of sleep intervals that were long enough to accommodate a full adult sleep cycle was very low for post-natal Month 2 but then improved:

* Month 2: 39%
* Month 3: 64%
* Month 4: 85%
* Month 5: 69%

This corresponded to at least one full adult sleep cycle occurring in the vast majority (98%) of nights overall.

However, the proportion of 24-hour periods that could accommodate at least the recommended four full sleep cycles was much lower:

* Month 2: 11% (4 nights)
* Month 3: 32% (10 nights)
* Month 4: 72% (23 nights)
* Month 5: 26% (8 nights)

**Conclusions**

Maternal sleep quality as measured by total sleep duration, number of sleep intervals, sleep interval durations, and number of full sleep cycles, improved steadily over post-natal Months 2, 3 and 4.

Despite there being at least one adult sleep cycle in 98% of nights, the proportion of nights in which there were at least four sleep cycles (as recommended by health experts) was low. In Month 2, only 4 nights could accommodate the recommended four adult 90-minute sleep cycles; in Months 3 and 5 the number was only 10 and 8 nights, respectively. Month 4 was the best month for getting good quality sleep, with 23 out of 31 nights having at least four full adult sleep cycles.

In post-natal Month 5, all measures of sleep quality began to decrease towards the values observed in Month 3. This indicates a baby sleep regression occured in Month 5, affecting maternal sleep.


# Descriptive statistics and plots

In [ ]:
# 1. Total sleep duration per 24 hours:

print( '\n*** Analysis of total sleep duration per 24 hours. ***\n' )

def print_summary( s , measure , unit):
  print( f"Summary data for {measure}:" )
  print( f"\tMedian = {s[ '50%' ]:.1f} {unit}" )
  print( f"\tMinimum = {s[ 'min' ]:.1f} {unit}" )
  print( f"\tMaximum = {s[ 'max' ]:.1f} {unit}" )
  print( "\n" )


# Overall result
tsd = df.groupby( 'daylabel' ).agg( { 'sleep_dur_hrs' : 'sum' } )
s = tsd[ "sleep_dur_hrs" ].describe()

print( "Over months 2 to 5, inclusive:" )
print_summary( s , "total sleep duration per 24 hours" , "hours" )

plt.boxplot( tsd[ 'sleep_dur_hrs' ] )
plt.xlabel( 'Months 2-5 inclusive' )
plt.ylabel( 'Total sleep (hours)' )
#plt.hist( tsd[ "sleep_dur_hrs" ] , bins=20 , histtype="bar")
plt.show()

# By month

print_summary( df[df['monthlabel'] == 'month2'].groupby( 'daylabel' ).agg( {'sleep_dur_hrs' : 'sum' } )[ 'sleep_dur_hrs' ].describe() ,
              "Month 2: total sleep duration per 24 hours" ,
               "hours" )

print_summary( df[df['monthlabel'] == 'month3'].groupby( 'daylabel' ).agg( {'sleep_dur_hrs' : 'sum' } )[ 'sleep_dur_hrs' ].describe() ,
              "Month 3: total sleep duration per 24 hours" ,
               "hours" )

print_summary( df[df['monthlabel'] == 'month4'].groupby( 'daylabel' ).agg( {'sleep_dur_hrs' : 'sum' } )[ 'sleep_dur_hrs' ].describe() ,
              "Month 4: total sleep duration per 24 hours" ,
               "hours" )

print_summary( df[df['monthlabel'] == 'month5'].groupby( 'daylabel' ).agg( {'sleep_dur_hrs' : 'sum' } )[ 'sleep_dur_hrs' ].describe() ,
              "Month 5: total sleep duration per 24 hours" ,
               "hours" )


In [ ]:
# 2. Number of sleep intervals per 24 hours:

print( "\n*** Number of sleep intervals per 24 hours***\n" )

tsd2 = df.groupby( 'daylabel' ).agg( { 'sleep_dur_hrs' : 'count' } )
s = tsd2[ "sleep_dur_hrs" ].describe()

print( "Over months 2 to 5, inclusive:" )
print_summary( s , "number of sleep intervals per 24 hours" , "sleep intervals" )

plt.hist( tsd2 , bins=13, histtype='bar' , edgecolor='black' )
plt.xlabel( 'Number of sleep intervals per 24 hours (Months 2-5 inclusive)' )
plt.ylabel( 'Frequency' )
plt.show()

# Analysis by month
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(nrows=2, ncols=2)
colors = [ 'indigo' , 'purple' , 'orchid'  , 'thistle' ]

df_month2 = df[df['monthlabel']=='month2']
df_month3 = df[df['monthlabel']=='month3']
df_month4 = df[df['monthlabel']=='month4']
df_month5 = df[df['monthlabel']=='month5']


ax0.hist( df_month2.groupby( 'daylabel' ).size() , 13, edgecolor='black', color=colors[0] )
ax0.set_title(f"Month 2: median = {df_month2.groupby( 'daylabel' ).size().median():.0f}")

ax1.hist( df_month3.groupby( 'daylabel' ).size() , 13, edgecolor='black' , color=colors[1] )
ax1.set_title(f"Month 3: median = {df_month3.groupby( 'daylabel' ).size().median():.0f}")

ax2.hist( df_month4.groupby( 'daylabel' ).size() , 13, edgecolor='black' , color=colors[2] )
ax2.set_title(f"Month 4: median = {df_month4.groupby( 'daylabel' ).size().median():.0f}")

ax3.hist( df_month5.groupby( 'daylabel' ).size() , 13, edgecolor='black' , color=colors[3] )
ax3.set_title(f"Month 5: median = {df_month5.groupby( 'daylabel' ).size().median():.0f}")

fig.tight_layout()
plt.show()


In [ ]:
# 3. Sleep interval durations (by month):

print( "\n*** Duration of individual sleep intervals (by month) ***\n" )

fig2, ((ax4, ax5), (ax6, ax7)) = plt.subplots(nrows=2, ncols=2)

ax4.hist( df_month2['sleep_dur_hrs'] , 13, edgecolor='black', color=colors[0] )
ax4.plot( [ 1.5 , 1.5 ] , [ 0 , 40 ] )
ax4.set_title(f"Month 2: median = {df_month2['sleep_dur_hrs'].median():.1f} hours")

ax5.hist( df_month3['sleep_dur_hrs'] , 13, edgecolor='black' , color=colors[1] )
ax5.plot( [ 1.5 , 1.5 ] , [ 0 , 20 ] )
ax5.set_title(f"Month 3: median = {df_month3['sleep_dur_hrs'].median():.1f} hours")

ax6.hist( df_month4['sleep_dur_hrs'] , 13, edgecolor='black' , color=colors[2] )
ax6.plot( [ 1.5 , 1.5 ] , [ 0 , 20 ] )
ax6.set_title(f"Month 4: median = {df_month4['sleep_dur_hrs'].median():.1f} hours")

ax7.hist( df_month5['sleep_dur_hrs'] , 13, edgecolor='black' , color=colors[3] )
ax7.plot( [ 1.5 , 1.5 ] , [ 0 , 20 ] )
ax7.set_title(f"Month 5: median = {df_month5['sleep_dur_hrs'].median():.1f} hours")

fig2.tight_layout()
plt.show()

# What proportion of sleep intervals were long enough to accommodate a full sleep cycle?

print( '\nQuestion 1: What proportion of sleep intervals were long enough to accommodate a full sleep cycle?' )
print( f"Month 2: {df_month2[ df_month2[ 'sleep_dur_hrs'] > 1.5 ]['sleep_dur_hrs'].count() / df_month2['sleep_dur_hrs'].count() *100:.0f}%" )
print( f"Month 3: {df_month3[ df_month3[ 'sleep_dur_hrs'] > 1.5 ]['sleep_dur_hrs'].count() / df_month3['sleep_dur_hrs'].count() *100:.0f}%" )
print( f"Month 4: {df_month4[ df_month4[ 'sleep_dur_hrs'] > 1.5 ]['sleep_dur_hrs'].count() / df_month4['sleep_dur_hrs'].count() *100:.0f}%" )
print( f"Month 5: {df_month5[ df_month5[ 'sleep_dur_hrs'] > 1.5 ]['sleep_dur_hrs'].count() / df_month5['sleep_dur_hrs'].count() *100:.0f}%" )

# What proportion of nights accommodated at least one full sleep cycle interval length?
df_gb2 = df_month2.groupby( 'daylabel' ).agg( { 'num_cycles' : 'sum' } )
df_gb3 = df_month3.groupby( 'daylabel' ).agg( { 'num_cycles' : 'sum' } )
df_gb4 = df_month4.groupby( 'daylabel' ).agg( { 'num_cycles' : 'sum' } )
df_gb5 = df_month5.groupby( 'daylabel' ).agg( { 'num_cycles' : 'sum' } )
df_gb = df.groupby( 'daylabel' ).agg( { 'num_cycles' : 'sum' } )

print( '\nQuestion 2: What proportion of nights accommodated at least one full sleep cycle interval length?' )
print( f"Month 2: {df_gb2[ df_gb2['num_cycles'] >= 1 ].size / df_month2['daylabel'].unique().size *100 :.0f}%" )
print( f"Month 3: {df_gb3[ df_gb3['num_cycles'] >= 1 ].size / df_month3['daylabel'].unique().size *100 :.0f}%" )
print( f"Month 4: {df_gb4[ df_gb4['num_cycles'] >= 1 ].size / df_month4['daylabel'].unique().size *100 :.0f}%" )
print( f"Month 5: {df_gb5[ df_gb5['num_cycles'] >= 1 ].size / df_month5['daylabel'].unique().size *100 :.0f}%" )
print( f"Overall: {df_gb[ df_gb['num_cycles'] >= 1 ].size / df['daylabel'].unique().size *100 :.0f}%" )



# What proportion of nights accommodated at least four full sleep cycles (minimum recommended amount for health)?
print( '\nQuestion 3: What proportion of nights accommodated at least four full sleep cycles (minimum recommended for healthy sleep)?' )
ndays2 = df_gb2[ df_gb2['num_cycles'] >= 4 ].size
ndays3 = df_gb3[ df_gb3['num_cycles'] >= 4 ].size
ndays4 = df_gb4[ df_gb4['num_cycles'] >= 4 ].size
ndays5 = df_gb5[ df_gb5['num_cycles'] >= 4 ].size

print( f"Month 2: {ndays2 / df_month2['daylabel'].unique().size *100 :.0f}% ({ndays2} nights)")
print( f"Month 3: {ndays3 / df_month3['daylabel'].unique().size *100 :.0f}% ({ndays3} nights)")
print( f"Month 4: {ndays4 / df_month4['daylabel'].unique().size *100 :.0f}% ({ndays4} nights)")
print( f"Month 5: {ndays5 / df_month5['daylabel'].unique().size *100 :.0f}% ({ndays5} nights)")


